# 한글 폰트 설치

In [1]:
# 폰트 설치
!apt-get update -qq # 나눔고딕 설치
!apt-get install fonts-nanum* -qq

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 폰트 로딩
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
plt.rc('font', family='NanumBarunGothic') 
mpl.font_manager._rebuild()

zsh:1: command not found: apt-get
zsh:1: no matches found: fonts-nanum*


# Install

In [ ]:
!pip install sentencepiece

# Evn

In [ ]:
import os
import random
import shutil
import json
import zipfile
import math
import copy
import collections
import re

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sentencepiece as spm
import tensorflow as tf
import tensorflow.keras.backend as K

from tqdm.notebook import tqdm

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
# random seed initialize
random_seed = 1234
random.seed(random_seed)
np.random.seed(random_seed)
tf.random.set_seed(random_seed)

In [ ]:
!nvidia-smi

# Word2Vec (Skip-gram)

# 1. 모델링

In [ ]:
# 입력 문장
sentences = """나는 오늘 기분이 좋아
나는 오늘 우울해"""

# 문장을 띄어쓰기 단위로 분할
words = sentences.split()

# 중복 단어 제거
words = list(dict.fromkeys(words))

# 각 단어별 고유한 번호 부여
word_to_id = {'[PAD]': 0, '[UNK]': 1}
for word in words:
    word_to_id[word] = len(word_to_id)
word_to_id

In [ ]:
# 입력(c), 출력(o) 정의
inputs = np.array([word_to_id['나는']])
labels = np.array([word_to_id['기분이']])
inputs, labels

In [ ]:
# center word embedding
V = tf.keras.layers.Embedding(len(word_to_id), 4)
v = V(inputs)
v

In [ ]:
# weight of center matrix
V.get_weights()

In [ ]:
# outer word embedding transpose
U = tf.keras.layers.Dense(len(word_to_id), use_bias=False)
vu = U(v)
vu

In [ ]:
# weight of outer matrix transpose
U.get_weights()[0].shape

In [ ]:
WU = U.get_weights()[0]
np.dot(v, WU[:,1])

In [ ]:
# exp(uv) / sum(exp(uv))
vu_prob = tf.nn.softmax(vu)
vu_prob

In [ ]:
# outer word embedding transpose & softmax
U = tf.keras.layers.Dense(len(word_to_id), use_bias=False, activation=tf.nn.softmax)
vu_prob = U(v)
vu_prob

In [ ]:
U.get_weights()[0].shape

In [ ]:
# label one hot
vu_label = tf.one_hot(labels, len(word_to_id))
vu_label

In [ ]:
# cross entroyp loss
loss = - vu_label * tf.math.log(vu_prob)
loss

In [ ]:
# sparse_categorical_crossentropy
tf.keras.losses.sparse_categorical_crossentropy(labels, vu_prob)

In [ ]:
def build_model(n_vocab, d_model):
    """
    skim gram 학습 모델
    :param n_vocab: vocabulary 단어 수
    :param d_model: 단어를 의미하는 벡터의 차원 수
    """
    tokens = tf.keras.layers.Input(shape=(1,))

    # center word vector
    V = tf.keras.layers.Embedding(n_vocab, d_model)
    v = V(tokens)
    # 단어 예측 (activation=tf.nn.softmax)
    U = tf.keras.layers.Dense(n_vocab, use_bias=False, activation=tf.nn.softmax)
    vu_prob = U(v)

    model = tf.keras.Model(inputs=tokens, outputs=vu_prob)
    return model, V

In [ ]:
# 모델 생성
model, V = build_model(len(word_to_id), 32)
# 모델 내용 그래프 출력
tf.keras.utils.plot_model(model, 'model.png', show_shapes=True)

# 2. 데이터

In [ ]:
# 학습할 말뭉치
text = """수학은 수식이 복잡해서 어렵다
수학은 공식이 많아서 어렵다
수학은 수식이 이해되면 쉽다
수학은 공식이 능통하면 쉽다
영어는 단어가 많아서 어렵다
영어는 듣기가 복잡해서 어렵다
영어는 단어가 이해되면 쉽다
영어는 듣기가 능통하면 쉽다
국어는 지문이 복잡해서 어렵다
국어는 한문이 많아서 어렵다
국어는 지문이 이해되면 쉽다
국어는 한문이 능통하면 쉽다"""

# 3. Vocabulary

In [ ]:
# 띄어쓰기 기준 단어 목록
words = list(dict.fromkeys(text.split()))
words

In [ ]:
# 각 단어별 고유한 번호 부여
word_to_id = {'[PAD]': 0, '[UNK]': 1}
for word in words:
    word_to_id[word] = len(word_to_id)
word_to_id

In [ ]:
# 각 숫자별 단어 부여
id_to_word = {_id:word for word, _id in word_to_id.items()}
id_to_word

# 4. 학습용 데이터 생성

In [ ]:
# 줄바꿈 단위로 문장 분리
sentences = text.split("\n")
sentences

In [ ]:
# 띄어쓰기 단위로 단어 분리
tokens = []
for sentence in sentences:
    tokens.append(sentence.split())
tokens

In [ ]:
# center-outer 생성
window_size = 2
word_pairs = []
for line_token in tokens:
    for i in range(len(line_token)):
        o_1 = max(0, i - window_size)
        o_2 = min(len(line_token) - 1, i + window_size)
        c = line_token[i]
        word_pair = {"c": c, "o": [line_token[j] for j in range(o_1, o_2 + 1) if j != i]}
        word_pairs.append(word_pair)
print(len(word_pairs))
word_pairs

In [ ]:
# skip gram dataset 생성
train_tokens = []
train_labels = []
for word_pair in word_pairs:
    c = word_pair["c"]
    o = word_pair["o"]
    for w in o:
        # center word 입력
        train_tokens.append(c)
        # outer word 정답
        train_labels.append(w)
print(f"tokens : {train_tokens}")
print(f"labels : {train_labels}")

In [ ]:
# input token to id
train_token_ids = np.array([word_to_id[token] for token in train_tokens])
train_token_ids

In [ ]:
# label token to id
train_label_ids = np.array([word_to_id[label] for label in train_labels])
train_label_ids

# 5. embedding 출력

In [ ]:
def plot_embdeeding(embedding, word_to_id):
    """
    word의 embedding vector를 2차원 공간에서 위치를 표현 함
    :param embedding: tf.keras.layers.Embedding 객체
    :param word_to_id: word_to_id vocab
    """
    # 폰트
    font_name = "NanumBarunGothic"

    # plot 크기 및 폰트 설정
    plt.figure(figsize=(8, 8))
    plt.rc('font', family=font_name)
    plt.rcParams["axes.unicode_minus"] = False # 한글 폰트 사용시 - 깨지는 문제 해결

    # word와 vector 값 추출
    vectors = []
    words = []
    for word, id in word_to_id.items():
        if id < 2: continue  # 0: PAD, 1: UNK
        vectors.append(embedding(id).numpy())
        words.append(word)

    # 2차원 보다 큰 경우 PCA를 이용해 2차원으로 차원 축소
    if 2 < len(vectors[0]):
        vectors = PCA().fit_transform(vectors)[:,:2]

    # 벡터와 단어를 화면에 출력
    for word, vector in zip(words, vectors):
        plt.scatter(vector[0], vector[1])
        plt.annotate(word, xy=(vector[0], vector[1]), xytext=(6, 4), textcoords='offset points', ha='right', va='bottom')

    # 출력
    plt.show()

# 6. 학습

In [ ]:
# 모델 loss, optimizer, metric 정의
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
plot_embdeeding(V, word_to_id)

In [ ]:
epochs = 100
# 20번 반복 진행
for i in range(20):
    # epoch 학습 진행 후 vector를 화면에 출력
    model.fit(train_token_ids, train_label_ids, batch_size=512, epochs=epochs, verbose=0)
    print(f"training >>> {(i+1) * epochs}")
    plot_embdeeding(V, word_to_id)